In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("SleepStudyData.csv")

df.insert(0, "Tired", df.pop("Tired"))

df["Enough"] = df["Enough"].map({"Yes":1, "No":0})
df["PhoneReach"] = df["PhoneReach"].map({"Yes":1, "No":0})
df["PhoneTime"] = df["PhoneTime"].map({"Yes":1, "No":0})
df["Breakfast"] = df["Breakfast"].map({"Yes":1, "No":0})

df = df.dropna()

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Input, Output, html, dcc
# from dash.dependencies import Input, Output, html, dcc
# import dash_core_components as dcc
# import dash_html_components as html
import dash_bootstrap_components as dbc


app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])
mytitle = dcc.Markdown(children='')
barplot = dcc.Graph(figure={})
dropdown = dcc.Dropdown(options=df.drop("Tired", axis=1).columns.values, 
                        value='Enough',
                        clearable =False)

app.layout = dbc.Container([
    html.H4("Graph"),
    barplot,
    dropdown,
])

# Callback allows components to interact
@app.callback(
    Output(barplot, 'figure'),
    Input(dropdown, 'value')
)
def update_graph(column_name):  # function arguments come from the component property of the Input
    
    fig = px.bar(df, x=column_name, y="Tired", barmode="group")
    return fig

# Run app

if __name__ == '__main__':
    #app.run_server(mode='inline', debug=False, use_reloader=False) 
    app.run_server(mode='inline', debug=True) 


In [ ]:
import seaborn as sns

sns.jointplot(x="Hours", y="Tired", data=df, kind="hex")
sns.jointplot(x="Hours", y="Tired", data=df, kind="kde")
sns.jointplot(x="Hours", y="Tired", data=df, kind="reg")

In [ ]:
import qgrid

qgrid.show_grid(df, grid_options={"editable": False, "enableColumnReorder":True, "forceFitColumns":False})


In [27]:
from sklearn.model_selection import train_test_split
df["Tired"] = df["Tired"].map({1:0, 2:0, 3:1, 4:1, 5:1})

X = df.drop("Tired", axis = 1)
y = df["Tired"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [28]:
from sklearn.linear_model import LogisticRegression
import numpy as np

np.random.seed(42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [29]:
model.score(X_test,y_test)

0.6666666666666666

In [30]:
from statsmodels.discrete.discrete_model import Logit
print(Logit(y, X).fit().summary())

Optimization terminated successfully.
         Current function value: 0.530076
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Tired   No. Observations:                  102
Model:                          Logit   Df Residuals:                       97
Method:                           MLE   Df Model:                            4
Date:                Sat, 21 May 2022   Pseudo R-squ.:                  0.1250
Time:                        08:57:03   Log-Likelihood:                -54.068
converged:                       True   LL-Null:                       -61.791
Covariance Type:            nonrobust   LLR p-value:                  0.003858
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Enough        -1.7883      0.524     -3.414      0.001      -2.815      -0.762
Hours          0.2462      0.

It appears that of the five factors that affect sleep, 